You can use this template for problems 1, 2 and 3

In [5]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

In [66]:
class AE20B108_Q3:
  def __init__(self):
    self.__num_actions = 6
    self.__alpha = 3
    self.__M_X = 1
    self.__M_R = 6
    self.__prev_action = None
    self.__arm_pulls = np.zeros(self.__num_actions)
    self.__reward = np.zeros(self.__num_actions)
    self.__cost = np.zeros(self.__num_actions)
    self.__V_R = np.zeros(self.__num_actions)
    self.__V_X = np.zeros(self.__num_actions)
    self.__n = 0

  def get_action(self, wicket, runs_scored):
    if not self.__prev_action is None:
      self.__arm_pulls[self.__prev_action] += 1
      self.__reward[self.__prev_action] += runs_scored
      self.__cost[self.__prev_action] += wicket
      self.__V_R[self.__prev_action] += runs_scored ** 2
      self.__V_X[self.__prev_action] += (wicket) ** 2
    if self.__n < self.__num_actions:
      action = self.__n
    else:
      r_mean = self.__reward / self.__arm_pulls
      X_mean = (self.__cost + 1e-15) / self.__arm_pulls
      R_var = self.__V_R / self.__arm_pulls - (self.__reward / self.__arm_pulls) ** 2
      X_var = self.__V_X / self.__arm_pulls - (self.__cost / self.__arm_pulls) ** 2
      eps = np.sqrt(2 * R_var * np.log(self.__n ** self.__alpha) / self.__arm_pulls) + 3 * self.__M_R * np.log(self.__n ** self.__alpha) / self.__arm_pulls
      eta = np.sqrt(2 * X_var * np.log(self.__n ** self.__alpha) / self.__arm_pulls) + 3 * self.__M_X * np.log(self.__n ** self.__alpha) / self.__arm_pulls
      rho_hat = r_mean / X_mean
      c_hat = 1.4 * (eps + rho_hat* eta) / X_mean
      action = np.argmax(rho_hat + c_hat)
    self.__n += 1
    self.__prev_action = action
    return action


In [26]:
class AE20B108_Q3:
  def __init__(self):
    self.__num_actions = 6
    self.__alpha = 3
    self.__M_X = 1
    self.__M_R = 6
    self.__previous_action = None
    self.__arm_pulls = np.zeros(self.__num_actions)
    self.__reward = np.zeros(self.__num_actions)
    self.__cost = np.zeros(self.__num_actions)
    self.__V_R = np.zeros(self.__num_actions)
    self.__V_X = np.zeros(self.__num_actions)
    self.__n = 0

  def get_action(self, wicket, runs_scored):
    if not self.__previous_action is None :
      self.__arm_pulls[self.__previous_action] += 1
      self.__reward[self.__previous_action] += runs_scored
      self.__cost[self.__previous_action] += (wicket)
      self.__V_R[self.__previous_action] += runs_scored ** 2
      self.__V_X[self.__previous_action] += wicket ** 2
    if self.__n < self.__num_actions:
      action = self.__n
    else:
      r_mean = self.__reward/self.__arm_pulls
      X_mean = (self.__cost + 1e-15)/self.__arm_pulls
      Var_R = self.__V_R/self.__arm_pulls - r_mean**2
      Var_X = self.__V_X/self.__arm_pulls - (self.__cost/self.__arm_pulls)**2
      eps = np.sqrt(2*Var_R*np.log(self.__n**self.__alpha)/self.__arm_pulls) + 3*self.__M_R*np.log(self.__n**self.__alpha)/self.__arm_pulls
      eta = np.sqrt(2*Var_X*np.log(self.__n**self.__alpha)/self.__arm_pulls) + 3*self.__M_X*np.log(self.__n**self.__alpha)/self.__arm_pulls
      rho_hat = r_mean/X_mean
      c_hat = 1.4*(eps + rho_hat*eta) / X_mean
      action = np.argmax(rho_hat + c_hat)
    self.__n += 1
    self.__previous_action = action
    return action

In [27]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__regret_w = 0
    self.__regret_s = 0
    self.__wicket = 0
    self.__regret_rho = 0
    self.__p_out = np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_run = np.array([1,0.9,0.85,0.8,0.75,0.7])
    # self.__p_out = np.array([0.04,0.1,0.2,0.3,0.5,0.7])
    # self.__p_run = np.array([1,0.9,0.55,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__s = (1-self.__p_out)*self.__p_run*self.__action_runs_map
    self.__rho = self.__s/self.__p_out


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket,self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun= self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      action = self.__get_action()
      self.__wicket, self.__runs_scored   = self.__get_outcome(action)
      self.__total_runs     = self.__total_runs + self.__runs_scored
      self.__total_wickets  = self.__total_wickets + self.__wicket
      self.__regret_w       = self.__regret_w+ (self.__p_out[action]-np.min(self.__p_out))
      self.__regret_s       = self.__regret_s+ (np.max(self.__s) - self.__s[action])
      self.__regret_rho       = self.__regret_rho+ (np.max(self.__rho)-self.__rho[action])
    return self.__regret_w,self.__regret_s,self.__regret_rho, self.__total_runs, self.__total_wickets, self.__run_time

In [41]:
agent = AE20B108_Q3()
environment = Environment(100,agent)
regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time = environment.innings()

In [42]:
print(regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)

1.800999999999992 173.1110000000003 556.0 116 3 0.006255149841308594
